In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import cv2
import warnings
warnings.filterwarnings('ignore')

In [16]:
# Load the CSV file
bbox_file = pd.read_csv("csv_files/BBox_List_2017.csv")

In [17]:
# Extract the image filenames, bounding box coordinates, and labels for Mass and Nodule findings
bbox_file_mass_nodule = bbox_file[(bbox_file['Finding Label'] == 'Mass') | (bbox_file['Finding Label'] == 'Nodule')]
bbox_file_mass_nodule['Image Index'] = 'chest_xrays/' + bbox_file_mass_nodule['Image Index'].astype(str)
image_files = bbox_file_mass_nodule['Image Index'].values
bbox_coords = bbox_file_mass_nodule[['Bbox [x', 'y', 'w', 'h]']].values
labels = bbox_file_mass_nodule['Finding Label'].values
display(bbox_file_mass_nodule)

,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8
583,chest_xrays/00023075_033.png,Mass,239.502222,535.077934,72.817778,65.991111,NaN,NaN,NaN
584,chest_xrays/00029579_005.png,Mass,609.280000,189.193490,73.955556,71.680000,NaN,NaN,NaN
585,chest_xrays/00013659_019.png,Mass,559.217778,167.575712,102.400000,136.533333,NaN,NaN,NaN
586,chest_xrays/00010815_006.png,Mass,311.182222,241.531267,146.773333,256.000000,NaN,NaN,NaN
587,chest_xrays/00026695_000.png,Mass,341.902222,197.157934,117.191111,131.982222,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
742,chest_xrays/00013674_000.png,Nodule,210.217989,319.661376,58.514286,57.430688,NaN,NaN,NaN
743,chest_xrays/00013751_003.png,Nodule,676.165079,610.065608,89.938624,89.938624,NaN,NaN,NaN
744,chest_xrays/00010103_014.png,Nodule,343.500529,412.850794,41.176720,45.511111,NaN,NaN,NaN
745,chest_xrays/00011576_000.png,Nodule,206.967196,582.975661,46.594709,41.176720,NaN,NaN,NaN


In [18]:
for i, image_file in enumerate(image_files[:10]):
    img = cv2.imread(image_file)
    if img is None:
        print(f"Could not read image file {image_file}")
        continue
    x, y, w, h = bbox_coords[i]
    img_bbox = cv2.rectangle(img, (int(x), int(y)), (int(x + w), int(y + h)), (255, 0, 0), 2)
    cv2.imshow("Image with Bounding Box", img_bbox)
    cv2.waitKey(0)
cv2.destroyAllWindows()

Could not read image file chest_xrays/00029579_005.png


In [19]:
# Preprocess the bounding box coordinates
bbox_coords = np.divide(bbox_coords, 1024) # Normalize the coordinates to be between 0 and 1

In [20]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(image_files, bbox_coords, test_size=0.2, random_state=42)

In [21]:
# Define the model architecture
model = keras.Sequential([
    keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224,224,3)),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(4, activation='linear')
])

In [22]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [23]:
import cv2
import numpy as np

def data_generator(image_files, bbox_coords, batch_size=32):
    while True:
        batch_indices = np.random.choice(len(image_files), size=batch_size)
        batch_files = image_files[batch_indices]
        batch_coords = bbox_coords[batch_indices]
        batch_images = np.zeros((batch_size, 224, 224, 3))
        batch_bbox = np.zeros((batch_size, 4))
        i = 0
        while i < len(batch_files):
            # Load the image
            img = cv2.imread(batch_files[i])
            if img is None:
                # Remove the row without an image path
                batch_files = np.delete(batch_files, i, axis=0)
                batch_coords = np.delete(batch_coords, i, axis=0)
                continue
            # Resize the image
            img = cv2.resize(img, (224,224))
            # Normalize the image pixels
            img = img / 255.0
            batch_images[i] = img
            batch_bbox[i] = batch_coords[i]
            i += 1
        yield batch_images[:i], batch_bbox[:i]

In [24]:
batch_size = 4
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_val) // batch_size
data_generator_train = data_generator(X_train, y_train, batch_size)
data_generator_val = data_generator(X_val, y_val, batch_size)
model.fit(data_generator_train, epochs=20, steps_per_epoch=steps_per_epoch, validation_data=data_generator_val, validation_steps=validation_steps)

Epoch 1/20
32/32 [==============================] - 4s 111ms/step - loss: 0.4799 - val_loss: 0.0466
Epoch 2/20
32/32 [==============================] - 3s 100ms/step - loss: 0.0317 - val_loss: 0.0392
Epoch 3/20
32/32 [==============================] - 3s 95ms/step - loss: 0.0302 - val_loss: 0.0282
Epoch 4/20
32/32 [==============================] - 3s 94ms/step - loss: 0.0252 - val_loss: 0.0392
Epoch 5/20
32/32 [==============================] - 3s 98ms/step - loss: 0.0212 - val_loss: 0.0255
Epoch 6/20
32/32 [==============================] - 3s 93ms/step - loss: 0.0190 - val_loss: 0.0396
Epoch 7/20
32/32 [==============================] - 3s 89ms/step - loss: 0.0154 - val_loss: 0.0388
Epoch 8/20
32/32 [==============================] - 3s 93ms/step - loss: 0.0147 - val_loss: 0.0371
Epoch 9/20
32/32 [==============================] - 3s 91ms/step - loss: 0.0117 - val_loss: 0.0326
Epoch 10/20
32/32 [==============================] - 3s 97ms/step - loss: 0.0098 - val_loss: 0.0278
Epoch 1

In [25]:
# Save the model
model.save('bbox_model.h5')

In [26]:
# Load the model
model = keras.models.load_model('bbox_model.h5')

In [28]:
for i, image_file in enumerate(X_val[:10]):
    # Load the image
    img = cv2.imread(image_file)
    if img is None:
        print(f"Could not read image file {image_file}")
        continue
    # Preprocess the image
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    # Predict the bounding box coordinates using the model
    bbox_coords = model.predict(np.array([img]))
    # Denormalize the bounding box coordinates
    bbox_coords = np.multiply(bbox_coords, 256)
    print(bbox_coords)
    # Load the ground truth bounding box coordinates for the image
    gt_bbox_coords = y_val[i]
    # Denormalize the ground truth bounding box coordinates
    gt_bbox_coords = np.multiply(gt_bbox_coords, 256)
    # Draw the predicted bounding box on the image in blue color
    x, y, w, h = bbox_coords[0]
    img_bbox = cv2.rectangle(img, (int(x), int(y)), (int(x + w), int(y + h)), (255, 0, 0), 2)
    # Draw the ground truth bounding box on the image in green color
    x, y, w, h = gt_bbox_coords
    img_bbox = cv2.rectangle(img_bbox, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)
    # Display the image
    cv2.namedWindow("Image with Bounding Box", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Image with Bounding Box", 800, 800)
    cv2.imshow("Image with Bounding Box", img_bbox)
    cv2.waitKey(0)
cv2.destroyAllWindows()

[[160.57014   129.27556    10.8299265   3.212947 ]]
[[108.88966  120.24906   19.158363   7.306674]]
Could not read image file chest_xrays/00030162_029.png
[[121.077255  92.01001   26.419643  10.144755]]
[[107.23047   81.5166    19.097984  36.34746 ]]
[[ 65.78336  159.1058     3.672576 -11.223302]]
[[94.37288  89.57743  30.156628 12.374827]]
[[132.68668  125.572014  24.532597  14.68146 ]]
Could not read image file chest_xrays/00030394_001.png
[[ 81.5696   117.050316  33.924076  33.07266 ]]
